In [4]:
import json
! pip install pypdf

In [11]:
! pip install -q -U google-generativeai

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

def split_pdf(pdfPath):
    loader = PyPDFLoader(pdfPath)
    data = loader.load()
    textSplitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    pages = textSplitter.split_documents(data)
    
    return pages

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
pages = split_pdf("Alice_in_Wonderland.pdf")
faiss_index = FAISS.from_documents(pages, embedding_function)
query = "Why does Alice go down the rabbit hole?"

docs = faiss_index.similarity_search(query, k=4)

In [10]:
prompt = f"For the following question: {query} , based on the story:\n {pages}"
print(prompt)

For the following question what is employee churn? based on following information [Document(metadata={'source': 'testDoc.pdf', 'page': 0}, page_content='https://doi.org/10.1007/s10844-020-00614-9\nAnovelschemeforemployeechurnproblemusing\nmulti-attributedecisionmakingapproach\nandmachinelearning\nNishantJain1·AbhinavTomar1·PrasantaK.Jana1\nReceived:28December2019/Revised:27July2020/Accepted:30July2020/\n©SpringerScience+BusinessMedia,LLC,partofSpringerNature2020\nAbstract\nEmployee churn (ECn) is a crucial problem for any organization that adversely affects its\noverall revenue and brand image. Many machine learning (ML) based systems have been\ndeveloped to solve the ECn problem. However, they miss out on some essential issues\nsuch as employee categorization, category-wise churn prediction, and retention policy for\neffectively addressing the ECn problem. By considering all these issues, we propose, in\nthis paper, a multi-attribute decision making (MADM) based scheme coupled with ML

In [13]:
API_KEY = "<KEY>"
def get_Gemini_Response(prompt):
    try:
        genai.configure(api_key=API_KEY)
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(e)
        return None